In [3]:
# imports 
from empath import Empath
lexicon = Empath()
import pandas as  pd
import matplotlib.pyplot as plt

## Steps for feature analysis aka violence metric


1. Take advantage of Empath package to obtain categories representatives of violence
    - a. Produce clean data locally thanks to results 
    - b. Preprocess as did in the paper to train Empath to ensure alignment with features found 
    - c. Perform Empath.lexicon.analyse with word counts, sorted_list (gives out cosine similarity between the vector of the category and the vector of the text given (need to reread that in paper))

2. Identify the most represented categories in the plot per genre, per year
3. Identify the most represented categories per movies classified as violent per ChatGPT 
4. Balance the frequencies of the words per their weight //TFID-matrix 
5. Our own logistic regression model : stantardise/normalise the train, test data 
6. Transform our plots in bags of words 
7. Side quest: Perform clustering on bags of words (idea: does it appear by itself group of violent movies)
8. Visualisation of number of words per movies, per genres 
9. If skewed, try logarithm and see if it becomes gaussian -> fit with logarithm 
10. Our own model : have p-value
Idea : try to replicate chatgpt predictions and explain them 

In [4]:
# Manual selection of hypothesised relevant categories 
categories = [
    "monster", "weapon", "negative_emotion", "pain", "terrorism", "breaking", 
    "disappointment", "anger", "torment", "shame", "emotional", "sadness", 
    "rage", "injury", "fire", "disgust", "war", "dominant_personality", 
    "fight", "deception", "neglect", "dominant_heirarchical", "violence"
]

for category in categories: 
    # Manual inspection of the words associated with all pre-build categories 
    print("Category =" + category, lexicon.cats.get(category))

Category =monster ['cursed', 'mutate', 'undead', 'werewolf', 'hunt', 'vicious', 'mythological', 'superpower', 'supernatural', 'devil', 'bloodthirsty', 'sighting', 'danger', 'underworld', 'vampire', 'immortal', 'hybrid', 'creature', 'beast', 'ferocious', 'foe', 'deadly', 'legendary', 'monstrous', 'evil', 'ravenous', 'mutant', 'inhuman', 'extinct', 'terrorize', 'mystical', 'myth', 'reincarnation', 'abomination', 'demonic', 'enemy', 'mutation', 'feral', 'monster', 'hunter', 'rogue', 'fearsome', 'alien', 'demon', 'witch', 'savage', 'feared', 'rabid', 'transformation', 'ruthless', 'wolf', 'fable', 'folklore', 'dragon', 'superhuman', 'parasite', 'unstoppable', 'predator', 'legend', 'mutated', 'extraterrestrial', 'mythical', 'villain', 'lore']
Category =weapon ['equip', 'gunfire', 'kill', 'rifle', 'artillery', 'marksman', 'machete', 'arrow', 'mechanism', 'deadly', 'loaded', 'slice', 'butcher', 'enemy', 'samurai', 'baton', 'pellet', 'atomic', 'arsenal', 'trigger', 'pistol', 'shrapnel', 'soldie

In [5]:
clean_data =pd.read_csv("../data/CLEAN/Movie_Data_clean.tsv",sep="\t")

# Genres extraction 

Analysis of error occuring when applying the genres extraction

In [8]:
df = clean_data.astype({'Genres':'string'}) # Ensure string type to perform string pre-processing
data_no_nan = clean_data[clean_data.Genres.isna()] # Movies that have no genres associated

In [9]:
print("There are {} movies that don't have a genre associated".format(len(data_no_nan)))

There are 13 movies that don't have a genre associated


In [116]:
def genres_extraction (y):
 test =[element for element in y.split(",")]
 return test


In [123]:
filtered_data =clean_data.Genres.dropna() # Keep only movies that have a genre 
genres = filtered_data.apply(genres_extraction)
from itertools import chain
merged_list = list(chain.from_iterable(genres.values))
unique_values =set(merged_list)
unique_genres = set([element.strip() for element in unique_values])
len(unique_genres)


333